In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes peft trl sentencepiece


In [ ]:
# ============================================
# STEP 1: Install Required Libraries (Fixed Versions)
# ============================================

# Option 1: Use latest compatible versions (RECOMMENDED)
!pip install -q transformers>=4.41.0 datasets accelerate bitsandbytes peft trl sentencepiece

# If Option 1 fails, try Option 2: Remove sentence-transformers conflict
# !pip uninstall -y sentence-transformers
# !pip install -q transformers==4.36.0 datasets accelerate bitsandbytes peft trl==0.7.11 sentencepiece

# Option 3: Use very latest versions (most compatible)
# !pip install -q --upgrade transformers datasets accelerate bitsandbytes peft trl sentencepiece

print("✅ Libraries installed successfully!")

In [ ]:
# STEP 2: Import Libraries
# ============================================
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import os

In [ ]:
from google.colab import files
uploaded = files.upload()  # select 20200325_counsel_chat.csv


In [ ]:
# STEP 3: Load and Prepare Your CSV Data
# ============================================
# Upload your CSV to Colab first
df = pd.read_csv('20200325_counsel_chat.csv')

# Create training prompts
def create_prompt(row):
    return f"""<|system|>
You are a compassionate mental health assistant trained by therapists. Provide empathetic, evidence-based guidance while encouraging professional help when needed.
<|user|>
{row['questionText']}
<|assistant|>
{row['answerText']}"""

df['text'] = df.apply(create_prompt, axis=1)

# Create dataset
dataset = Dataset.from_pandas(df[['text']])
train_test = dataset.train_test_split(test_size=0.1, seed=42)

In [ ]:
# STEP 4: Model Configuration (T4 GPU Optimized)
# ============================================
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Alternative: "microsoft/phi-2" Excellent for T4, 2.7B params


# Quantization config for T4 GPU
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# STEP 5: LoRA Configuration (Parameter-Efficient)
# ============================================
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "dense"]
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

print(f"Trainable parameters: {model.print_trainable_parameters()}")


In [ ]:
# ============================================
# STEP 6: Training Configuration (Fixed for Transformers Trainer)
# ============================================

# Since we're using standard Transformers Trainer, use compatible args
training_args = TrainingArguments(
    output_dir="./mental-health-model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    bf16=True,    # Use bf16 for better stability
    save_strategy="epoch",
    eval_strategy="epoch",  # This will work with eval_dataset
    logging_steps=10,
    warmup_steps=100,
    max_grad_norm=0.3,
    group_by_length=True,
    lr_scheduler_type="cosine",
    dataloader_pin_memory=False,
    remove_unused_columns=True,  # Changed to True for standard trainer
    gradient_checkpointing=True,
    report_to="none",  # Explicitly disable wandb logging
)

print("✅ Training arguments created successfully!")

In [ ]:
# ============================================
# STEP 7: Initialize Trainer (TRL 0.23.0+ Compatible)
# ============================================

# Check versions for debugging
import trl, transformers
print(f"Transformers version: {transformers.__version__}")
print(f"TRL version: {trl.__version__}")

# For TRL 0.23.0+, the API has changed significantly
# Let's check what parameters SFTTrainer actually accepts
import inspect
sft_params = inspect.signature(SFTTrainer.__init__).parameters
print(f"SFTTrainer parameters: {list(sft_params.keys())}")

# Function to format the dataset for SFTTrainer
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['text'])):
        text = example['text'][i]
        output_texts.append(text)
    return output_texts

# TRL 0.23.0+ uses a different API - try the correct parameters
try:
    # Method 1: Latest TRL 0.23.0+ API
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_test["train"],
        eval_dataset=train_test["test"],
        processing_class=tokenizer,
        formatting_func=formatting_prompts_func,
        packing=False,
    )
    print("✅ Using TRL 0.23.0+ with processing_class")
except Exception as e1:
    print(f"Method 1 failed: {e1}")
    try:
        # Method 2: Try without formatting_func
        trainer = SFTTrainer(
            model=model,
            args=training_args,
            train_dataset=train_test["train"],
            eval_dataset=train_test["test"],
            processing_class=tokenizer,
            packing=False,
        )
        print("✅ Using TRL 0.23.0+ without formatting_func")
    except Exception as e2:
        print(f"Method 2 failed: {e2}")
        try:
            # Method 3: Minimal configuration
            trainer = SFTTrainer(
                model=model,
                args=training_args,
                train_dataset=train_test["train"],
                processing_class=tokenizer,
            )
            print("✅ Using TRL 0.23.0+ minimal config")
        except Exception as e3:
            print(f"Method 3 failed: {e3}")
            # Method 4: Try with dataset that has format expected by new TRL
            # Preprocess the dataset to the expected format
            def preprocess_dataset(dataset):
                def format_chat(example):
                    return {
                        "messages": [
                            {"role": "system", "content": "You are a compassionate mental health assistant trained by therapists. Provide empathetic, evidence-based guidance while encouraging professional help when needed."},
                            {"role": "user", "content": example["text"].split("<|user|>")[1].split("<|assistant|>")[0].strip()},
                            {"role": "assistant", "content": example["text"].split("<|assistant|>")[1].strip()}
                        ]
                    }
                return dataset.map(format_chat)

            try:
                formatted_train = preprocess_dataset(train_test["train"])
                formatted_eval = preprocess_dataset(train_test["test"])

                trainer = SFTTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=formatted_train,
                    eval_dataset=formatted_eval,
                    processing_class=tokenizer,
                )
                print("✅ Using TRL 0.23.0+ with formatted chat dataset")
            except Exception as e4:
                print(f"All methods failed: {e1}, {e2}, {e3}, {e4}")
                print("Falling back to standard Transformer Trainer...")

                # Fallback to standard Transformer Trainer
                from transformers import Trainer, DataCollatorForLanguageModeling

                # Tokenize the dataset
                def tokenize_function(examples):
                    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

                tokenized_train = train_test["train"].map(tokenize_function, batched=True)
                tokenized_eval = train_test["test"].map(tokenize_function, batched=True)

                data_collator = DataCollatorForLanguageModeling(
                    tokenizer=tokenizer,
                    mlm=False,
                )

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset=tokenized_train,
                    eval_dataset=tokenized_eval,
                    data_collator=data_collator,
                )
                print("✅ Using standard Transformers Trainer as fallback")

print("Trainer initialized successfully!")

In [ ]:
# Disable wandb logging
import os
os.environ["WANDB_DISABLED"] = "true"



In [ ]:
# Then continue with Step 8
print("🚀 Starting training...")
trainer.train()

In [ ]:
# STEP 9: Save the Model
# ============================================
# Save the fine-tuned model
model.save_pretrained("./mental-health-model-final")
tokenizer.save_pretrained("./mental-health-model-final")

print("✅ Training complete!")

In [ ]:
# ============================================
# STEP 10: Download the Model
# ============================================
# Zip the model for download
!zip -r mental-health-model-final.zip ./mental-health-model-final

# Download via Colab
from google.colab import files
files.download('mental-health-model-final.zip')

print("""
✅ TRAINING COMPLETE!

NEXT STEPS:
1. Download 'mental-health-model-final.zip' from Colab
2. Extract it to your local machine
3. Place the extracted folder in: 2_flask_app/trained_model/
4. Follow the Flask setup instructions
""")

In [ ]:
# ============================================
# OPTIONAL: Test the Model
# ============================================
def test_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test prompt
test_prompt = """<|system|>
You are a compassionate mental health assistant.
<|user|>
I've been feeling very anxious lately. What can I do?
<|assistant|>
"""

print("\n🧪 Testing the model:")
print(test_model(test_prompt))